In [45]:
import math

import numpy as np

In [67]:
def point(x, y):
    return np.array([x, y])

In [79]:
def rotate_point_clockwise(point, deg_rotation):
    rad_rotation = math.radians(deg_rotation)
    rotation_matrix = np.array([[math.cos(rad_rotation), math.sin(rad_rotation)], [-math.sin(rad_rotation), math.cos(rad_rotation)]])
    return np.dot(rotation_matrix, point)

# Rotate around the centroid of the polygon (average of all points)
def rotate_polygon_clockwise(points, deg_rotation):
    total_x = 0
    total_y = 0
    for point in points:
        total_x = total_x + point[0]
        total_y = total_y + point[0]
    center = np.array([total_x, total_y]) / len(points)
    # Now subtract the center point to get the shape centered about origin
    centered_points = [point - center for point in points]
    centered_points = [rotate_point_clockwise(point, deg_rotation) for point in centered_points]
    return [point + center for point in centered_points]


In [82]:
x = point(1,1)
rotate_polygon_clockwise([point(-1,1), point(1,1), point(1,-1), point(-1,-1)], -90)

[array([-1., -1.]), array([-1.,  1.]), array([1., 1.]), array([ 1., -1.])]

In [42]:
def ballistic_sim(initial_velocity, elevation, target_height, delta_t):
    t = 0
    x = 0
    y = 0
    v_0 = math.sin(math.radians(elevation)) * initial_velocity
    v_y = math.sin(math.radians(elevation)) * initial_velocity
    v_x = math.cos(math.radians(elevation)) * initial_velocity
    while v_y > 0 or y > target_height:
        t = t + delta_t
        x = x + v_x * delta_t
        y = y + v_y * delta_t
        v_y = v_0 - 9.81 * t
    return x, t


def ballistic_sim2(initial_velocity, elevation, target_height, delta_t):
    f = 0.00031  # from arma for 81mm
    t = 0
    x = 0
    y = 0
    v_y = math.sin(math.radians(elevation)) * initial_velocity
    v_x = math.cos(math.radians(elevation)) * initial_velocity
    while v_y > 0 or y > target_height:
        v = math.sqrt(v_x**2 + v_y**2)
        deceleration = f * v**2
        v_theta = math.atan(v_y / v_x)
        # decel_x = -1 * math.cos(v_theta) * deceleration
        # decel_y = -1 * math.sin(v_theta) * deceleration
        decel_x = -1 * deceleration
        decel_y = -1 * deceleration
        decel_y = decel_y - 9.81

        t = t + delta_t
        x = x + v_x * delta_t
        y = y + v_y * delta_t
        v_x = v_x + (decel_x * delta_t)
        v_y = v_y + (decel_y * delta_t)
    return x, t


def ballistic_sim3(initial_velocity, elevation, target_height, delta_t, include_drag):
    '''Calculates the distance and time of flight for a projectile launched
    with `initial_velocity` m/s, `elevation` degrees, landing at `target_height`
    relative height from launch height, with simulation timestep of `delta_t`
    '''
    plot_points = []
    f = 0.00031  # Air Drag of 81mm mortar shell
    m = .25
    t = 0
    position = np.array([0, 0])
    gravity = np.array([0, -9.81])
    velocity = initial_velocity * np.array(
        [math.cos(math.radians(elevation)), math.sin(math.radians(elevation))]
    )
    while velocity[1] > 0 or position[1] > target_height:
        plot_points.append(position)
        position = position + velocity * delta_t

        speed = np.linalg.norm(velocity)
        drag_speed = (f * speed * speed * m)
        drag_direction = -1 * (velocity / speed)
        drag_vector = drag_speed * drag_direction
        deceleration = drag_vector + gravity if include_drag else gravity

        velocity = velocity + deceleration * delta_t
        t = t + delta_t
    return position[0], t, plot_points


# FIXME: Not taking into account relative height from observer, which means I'm
# calculating eastings/northings based on hypotenuse of the triangle rather
# than the actual x/y distance
def azimuth_from_observer_relative(
    mortar_easting: float,
    mortar_northing: float,
    observer_easting: float,
    observer_northing: float,
    obs_to_enemy_azimuth: float,
    obs_to_enemy_distance: float,
):
    # First need to calculate easting and northing of enemy
    # relative to observer
    obs_to_enemy_easting = 0.0
    obs_to_enemy_northing = 0.0

    # Due North
    if obs_to_enemy_azimuth == 0:
        obs_to_enemy_easting = 0
        obs_to_enemy_northing = obs_to_enemy_distance
    # First Quadrant
    elif obs_to_enemy_azimuth < 90:
        angle = 90 - obs_to_enemy_azimuth
        obs_to_enemy_easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_northing = math.sin(math.radians(angle)) * obs_to_enemy_distance
    # Due East
    elif obs_to_enemy_azimuth == 90:
        obs_to_enemy_easting = obs_to_enemy_distance
        obs_to_enemy_northing = 0
    # Second Quadrant
    elif obs_to_enemy_azimuth < 180:
        angle = obs_to_enemy_azimuth - 90
        obs_to_enemy_easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due South
    elif obs_to_enemy_azimuth == 180:
        obs_to_enemy_easting = 0
        obs_to_enemy_northing = -1 * obs_to_enemy_distance

    # Third Quadrant
    elif obs_to_enemy_azimuth < 270:
        angle = 270 - obs_to_enemy_azimuth
        obs_to_enemy_easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due West
    elif obs_to_enemy_azimuth == 270:
        obs_to_enemy_easting = -1 * obs_to_enemy_distance
        obs_to_enemy_northing = 0
    # Fourth Quadrant
    else:
        angle = obs_to_enemy_azimuth - 270
        obs_to_enemy_easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_northing = math.sin(math.radians(angle)) * obs_to_enemy_distance

    enemy_easting = observer_easting + obs_to_enemy_easting
    enemy_northing = observer_northing + obs_to_enemy_northing

    distance = distancefunc(
        mortar_easting, mortar_northing, enemy_easting, enemy_northing
    )

    return (
        azimuth_from_grids(
            mortar_easting, mortar_northing, enemy_easting, enemy_northing
        ),
        distance,
    )


def distancefunc(x1, y1, x2, y2):
    return math.sqrt(math.pow((x2 - x1), 2) + math.pow((y2 - y1), 2))


def azimuth_from_grids(start_easting, start_northing, end_easting, end_northing):
    east_vector = end_easting - start_easting
    north_vector = end_northing - start_northing

    # since we want azimuth, we need to shift the axes
    # east becomes north, and north becomes west

    temp = north_vector
    north_vector = east_vector
    east_vector = temp

    initial = math.degrees(math.atan2(north_vector, east_vector))
    if initial < 0:
        initial = 360 + initial

    return initial


def enemy_pos_from_observer_relative(
    mortar_easting,
    mortar_northing,
    observer_easting,
    observer_northing,
    obs_to_enemy_azimuth,
    obs_to_enemy_distance,
):
    # First need to calculate easting and northing of enemy
    # relative to observer
    obs_to_enemy_easting = 0
    obs_to_enemy_northing = 0

    # Due North
    if obs_to_enemy_azimuth == 0:
        obs_to_enemy_easting = 0
        obs_to_enemy_northing = obs_to_enemy_distance
    # First Quadrant
    elif obs_to_enemy_azimuth < 90:
        angle = 90 - obs_to_enemy_azimuth
        obs_to_enemy_easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_northing = math.sin(math.radians(angle)) * obs_to_enemy_distance
    # Due East
    elif obs_to_enemy_azimuth == 90:
        obs_to_enemy_easting = obs_to_enemy_distance
        obs_to_enemy_northing = 0
    # Second Quadrant
    elif obs_to_enemy_azimuth < 180:
        angle = obs_to_enemy_azimuth - 90
        obs_to_enemy_easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due South
    elif obs_to_enemy_azimuth == 180:
        obs_to_enemy_easting = 0
        obs_to_enemy_northing = -1 * obs_to_enemy_distance

    # Third Quadrant
    elif obs_to_enemy_azimuth < 270:
        angle = 270 - obs_to_enemy_azimuth
        obs_to_enemy_easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due West
    elif obs_to_enemy_azimuth == 270:
        obs_to_enemy_easting = -1 * obs_to_enemy_distance
        obs_to_enemy_northing = 0
    # Fourth Quadrant
    else:
        angle = obs_to_enemy_azimuth - 270
        obs_to_enemy_easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_northing = math.sin(math.radians(angle)) * obs_to_enemy_distance

    enemy_easting = observer_easting + obs_to_enemy_easting
    enemy_northing = observer_northing + obs_to_enemy_northing
    return enemy_easting, enemy_northing


def calculate_elevation(
    initial_velocity: float, target_height: float, distance: float, upper_half: bool
):
    # Check if target is even reachable using optimal launch angle
    current_elevation = 45
    current_distance = ballistic_sim3(
        initial_velocity, current_elevation, target_height, 0.001, False
    )[0]
    if current_distance < distance:
        # target unreachable
        return HTMLResponse("Target Unreachable.", status_code=400)

    if upper_half:
        max_elevation = 90
        min_elevation = 45
    else:
        max_elevation = 45
        min_elevation = 0
    while abs(current_distance - distance) > 1:
        current_elevation = (min_elevation + max_elevation) / 2
        current_distance = ballistic_sim3(
            initial_velocity, current_elevation, target_height, 0.001, False
        )[0]
        if upper_half:
            if current_distance < distance:
                max_elevation = current_elevation
            else:
                min_elevation = current_elevation
    return {
        "elevation": round(current_elevation, 1),
        "time_to_impact": round(
            ballistic_sim3(initial_velocity, current_elevation, target_height, 0.001, False)[
                1
            ],
            1,
        ),
    }


def elevation_regression(distance):
    if distance > 1500:
        return -175
    else:
        return (
            603
            + -0.347 * distance
            + -1.28e-04 * distance**2
            + 1.64e-07 * distance**3
            + -1.03e-10 * distance**4
        )

In [43]:
def generate_ballistics_table(initial_velocity, air_drag, heights = None):
    if heights == None:
        heights = [height for height in range(0, -10000, -100)]
    else:
        heights = [-1 * height for height in heights]
    for height in heights:
        result = ballistic_sim3(initial_velocity, 0, height, .001, False)
        print("x: ", result[0], "y: -", height, "z: ", result[1])

In [44]:
reforger_tools_drops = [
    16.672001,
    37.368,
    65.996002,
    102.334999,
    123.328003,
    146.169006,
    170.830002,
    197.285004,
    225.509995,
    255.477997,
    287.165985,
    320.548004,
    355.600006,
    392.298004,
    430.618011,
    470.536987,
    512.031006,
    555.078979,
    599.656006,
    645.742004,
    693.315002,
    742.348999,
    792.828003,
    844.728027,
    898.026001,
    952.70697,
    1008.742981,
    1066.119019,
    1124.812988,
    1184.807983,
    1246.081055,
    1277.192993,
    1308.61499,
    1340.348022,
    1372.38501,
    1404.733032,
    1437.383057,
    1470.332031,
    1503.577026,
    1537.119995,
    1570.958008,
    1605.088013,
    1639.505981,
    1674.207031,
    1709.199951,
    1744.469971,
    1780.021973,
    1815.850952,
    1851.959961,
    1888.333984,
    1924.989014,
    1961.911011,
    1999.089966,
    2036.546021,
    2074.261963,
    2112.233887,
    2150.462891,
    2188.954102,
    2227.699951,
    2266.697021,
    2305.940918,
    2345.435059,
    2385.180908,
    2425.159912,
    2465.388916,
    2505.846924,
    2546.559082,
    2587.493896,
    2628.670898,
    2670.074951,
    2711.705078,
    2753.579102,
    2795.662109,
    2837.971924,
    2880.51001,
    2923.27002,
    2966.239014,
    3009.429932,
    3052.831055,
    3096.447021,
    3140.263916,
    3184.310059,
    3228.552979,
    3272.98999,
    3317.649902,
    3362.5,
    3407.548096,
    3452.795898,
    3498.24292,
    3543.878906,
    3589.702881,
    3635.727051,
    3681.937988,
    3728.331055,
    3774.907959,
    3821.674072,
    3868.616943,
    3915.731934,
    3963.039062,
    4010.506104,
    4058.163086,
    4105.987793,
    4153.984863,
    4202.146973,
    4250.487793,
    4298.966797,
    4347.626953,
    4396.454102,
    4445.434082,
    4494.575195,
    4543.878906,
    4593.331055,
    4642.938965,
    4692.696777,
    4742.609863,
    4792.666016,
    4842.868164,
    4893.221191,
    4943.708984,
    4994.356934,
    5045.134766,
    5096.048828,
    5147.094238,
    5198.279785,
    5223.944824,
    5249.606934,
    5301.062988,
    5352.658203,
    5404.36084,
    5456.212891,
    5508.169922,
    5560.282227,
    5612.487793,
    5664.821777,
    5717.266113,
    5769.845215,
    5822.554199,
    5875.347168,
    5928.262207,
    5981.300781,
    6034.458008,
    6087.696777,
    6141.054199,
    6194.508789,
    6248.080078,
    6274.911133,
    6301.745117,
    6355.512207,
    6409.395996,
    6463.367188,
    6517.424805,
    6571.588867,
    6598.713867,
    6625.838867,
    6680.192871,
    6734.625977,
    6789.143066,
    6816.444824,
    6871.108887,
    6925.845215,
    6953.23584,
    7008.10791,
    7063.030762,
    7090.568848,
    7118.070801,
    7173.173828,
    7200.762207,
    7256,
    7283.61377,
    7338.937988,
    7366.652832,
    7394.353027,
    7449.835938,
    7505.373047,
    7560.977051,
    7588.811035,
    7616.634766,
    7672.399902,
    7728.21582,
    7784.083008,
    7840.027832,
    7896.005859,
    7924.032227,
    7952.045898,
    8008.150879,
    8036.243164,
    8064.330078,
    8120.519043,
    8148.647949,
    8176.780762,
    8204.944336,
    8233.103516,
    8289.466797,
    8345.873047,
    8374.139648,
    8402.34082,
    8458.867188,
    8515.391602,
    8543.688477,
    8571.991211,
    8600.298828,
    8656.950195,
    8713.626953,
    8741.986328,
    8798.723633,
    8827.123047,
    8855.493164,
    8883.899414,
    8912.291992,
    8940.71875,
    8969.149414,
    9026.033203,
    9054.472656,
    9082.914062,
    9111.380859,
    9139.84668,
    9168.333984,
    9196.817383,
    9253.785156,
]

generate_ballistics_table(375, 0, reforger_tools_drops)

x:  691.875 y: - -16.672001 z:  1.8449999999999076
x:  1035.375 y: - -37.368 z:  2.760999999999807
x:  1375.875 y: - -65.996002 z:  3.668999999999707
x:  1713.375 y: - -102.334999 z:  4.5689999999998605
x:  1880.625 y: - -123.328003 z:  5.0150000000000095
x:  2047.5 y: - -146.169006 z:  5.460000000000158
x:  2213.625 y: - -170.830002 z:  5.903000000000306
x:  2378.625 y: - -197.285004 z:  6.343000000000453
x:  2543.25 y: - -225.509995 z:  6.7820000000005995
x:  2706.75 y: - -255.477997 z:  7.218000000000745
x:  2869.875 y: - -287.165985 z:  7.65300000000089
x:  3031.875 y: - -320.548004 z:  8.085000000000958
x:  3193.5 y: - -355.600006 z:  8.51600000000072
x:  3354.0 y: - -392.298004 z:  8.944000000000482
x:  3514.125 y: - -430.618011 z:  9.371000000000246
x:  3673.125 y: - -470.536987 z:  9.79500000000001
x:  3831.75 y: - -512.031006 z:  10.217999999999776
x:  3989.625 y: - -555.078979 z:  10.638999999999543
x:  4146.75 y: - -599.656006 z:  11.05799999999931
x:  4303.125 y: - -645.742

KeyboardInterrupt: 

679.153015	16.672001	1.837
1014.54303	37.368	2.764
1343.380981	65.996002	3.686
1665.703979	102.334999	4.603
1824.437988	123.328003	5.06
1981.560059	146.169006	5.516
2137.069092	170.830002	5.97
2290.954102	197.285004	6.424
2443.235107	225.509995	6.876
2593.912109	255.477997	7.326
2742.991943	287.165985	7.776
2890.477051	320.548004	8.224
3036.37207	355.600006	8.672
3180.665039	392.298004	9.117
3323.38208	430.618011	9.562
3464.51001	470.536987	10.005
3604.060059	512.031006	10.447
3742.040039	555.078979	10.888
3878.439941	599.656006	11.328
4013.274902	645.742004	11.766
4146.559082	693.315002	12.203
4278.263184	742.348999	12.638
4408.421875	792.828003	13.072
4537.025879	844.728027	13.505
4664.065918	898.026001	13.937
4789.582031	952.70697	14.367
4913.53418	1008.742981	14.796
5035.949219	1066.119019	15.224
5156.832031	1124.812988	15.65
5276.189941	1184.807983	16.075001
5394.01416	1246.081055	16.497999
5452.36084	1277.192993	16.709999
5510.315918	1308.61499	16.921
5567.897949	1340.348022	17.131001
5625.083008	1372.38501	17.341
5681.912109	1404.733032	17.551001
5738.353027	1437.383057	17.761
5794.413086	1470.332031	17.969999
5850.083008	1503.577026	18.179001
5905.384766	1537.119995	18.386999
5960.312988	1570.958008	18.596001
6014.866211	1605.088013	18.802999
6069.041016	1639.505981	19.011
6122.820801	1674.207031	19.218
6176.256836	1709.199951	19.424999
6229.293945	1744.469971	19.631001
6281.962891	1780.021973	19.837
6334.25293	1815.850952	20.042999
6386.187988	1851.959961	20.247999
6437.717773	1888.333984	20.452999
6488.905762	1924.989014	20.658001
6539.713867	1961.911011	20.862
6590.117188	1999.089966	21.066
6640.187012	2036.546021	21.27
6689.87793	2074.261963	21.473
6739.180176	2112.233887	21.676001
6788.108887	2150.462891	21.879
6836.686035	2188.954102	22.080999
6884.893066	2227.699951	22.283001
6932.727051	2266.697021	22.483999
6980.183105	2305.940918	22.684999
7027.273926	2345.435059	22.886
7074.019043	2385.180908	23.086
7120.35791	2425.159912	23.285999
7166.355957	2465.388916	23.486
7211.958008	2505.846924	23.684999
7257.233887	2546.559082	23.884001
7302.109863	2587.493896	24.083
7346.64209	2628.670898	24.281
7390.795898	2670.074951	24.479
7434.571777	2711.705078	24.677
7478.028809	2753.579102	24.874001
7521.075195	2795.662109	25.070999
7563.76416	2837.971924	25.267
7606.105957	2880.51001	25.462999
7648.087891	2923.27002	25.659
7689.684082	2966.239014	25.854
7730.934082	3009.429932	26.049
7771.808105	3052.831055	26.243999
7812.329102	3096.447021	26.438
7852.461914	3140.263916	26.632
7892.287109	3184.310059	26.825001
7931.725098	3228.552979	27.018
7970.774902	3272.98999	27.211
8009.516113	3317.649902	27.403
8047.870117	3362.5	27.594999
8085.85791	3407.548096	27.787001
8123.496094	3452.795898	27.978001
8160.782227	3498.24292	28.169001
8197.699219	3543.878906	28.360001
8234.250977	3589.702881	28.549999
8270.464844	3635.727051	28.74
8306.320312	3681.937988	28.929001
8341.804688	3728.331055	29.118
8376.933594	3774.907959	29.306999
8411.720703	3821.674072	29.495001
8446.140625	3868.616943	29.683001
8480.191406	3915.731934	29.870001
8513.915039	3963.039062	30.056999
8547.250977	4010.506104	30.243999
8580.264648	4058.163086	30.431
8612.915039	4105.987793	30.617001
8645.216797	4153.984863	30.802
8677.154297	4202.146973	30.987
8708.762695	4250.487793	31.172001
8739.963867	4298.966797	31.357
8770.84668	4347.626953	31.541
8801.386719	4396.454102	31.725
8831.55957	4445.434082	31.908001
8861.387695	4494.575195	32.091
8890.874023	4543.878906	32.272999
8919.995117	4593.331055	32.456001
8948.774414	4642.938965	32.637001
8977.195312	4692.696777	32.819
9005.277344	4742.609863	33
9032.998047	4792.666016	33.181
9060.369141	4842.868164	33.361
9087.399414	4893.221191	33.541
9114.068359	4943.708984	33.720001
9140.421875	4994.356934	33.898998
9166.405273	5045.134766	34.077999
9192.038086	5096.048828	34.256001
9217.311523	5147.094238	34.433998
9242.249023	5198.279785	34.612
9254.625977	5223.944824	34.700001
9266.852539	5249.606934	34.789001
9291.102539	5301.062988	34.966
9315.022461	5352.658203	35.141998
9338.55957	5404.36084	35.318001
9361.788086	5456.212891	35.493999
9384.638672	5508.169922	35.668999
9407.192383	5560.282227	35.844002
9429.353516	5612.487793	36.018002
9451.185547	5664.821777	36.192001
9472.655273	5717.266113	36.366001
9493.80957	5769.845215	36.539001
9514.643555	5822.554199	36.712002
9535.083984	5875.347168	36.883999
9555.197266	5928.262207	37.056
9574.987305	5981.300781	37.228001
9594.447266	6034.458008	37.398998
9613.524414	6087.696777	37.57
9632.277344	6141.054199	37.740002
9650.677734	6194.508789	37.91
9668.754883	6248.080078	38.080002
9677.676758	6274.911133	38.165001
9686.478516	6301.745117	38.249001
9703.864258	6355.512207	38.417999
9720.935547	6409.395996	38.587002
9737.652344	6463.367188	38.755001
9754.015625	6517.424805	38.922001
9770.056641	6571.588867	39.09
9777.961914	6598.713867	39.173
9785.748047	6625.838867	39.256001
9801.119141	6680.192871	39.423
9816.135742	6734.625977	39.589001
9830.807617	6789.143066	39.754002
9838.032227	6816.444824	39.837002
9852.22168	6871.108887	40.001999
9866.055664	6925.845215	40.167
9872.836914	6953.23584	40.249001
9886.183594	7008.10791	40.412998
9899.149414	7063.030762	40.576
9905.572266	7090.568848	40.658001
9911.832031	7118.070801	40.740002
9924.158203	7173.173828	40.903
9930.206055	7200.762207	40.984001
9942.057617	7256	41.146
9947.811523	7283.61377	41.227001
9959.125977	7338.937988	41.388
9964.692383	7366.652832	41.469002
9970.129883	7394.353027	41.549999
9980.796875	7449.835938	41.709999
9991.109375	7505.373047	41.870998
10001.08594	7560.977051	42.029999
10005.95996	7588.811035	42.110001
10010.71582	7616.634766	42.189999
10020.06641	7672.399902	42.348999
10029.06641	7728.21582	42.507999
10037.7207	7784.083008	42.666
10046.06641	7840.027832	42.824001
10054.04785	7896.005859	42.980999
10057.93262	7924.032227	43.060001
10061.70117	7952.045898	43.138
10069.03516	8008.150879	43.294998
10072.60156	8036.243164	43.373001
10076.06055	8064.330078	43.451
10082.7041	8120.519043	43.606998
10085.91992	8148.647949	43.683998
10089.04199	8176.780762	43.762001
10092.10352	8204.944336	43.84
10095.06445	8233.103516	43.917
10100.74316	8289.466797	44.070999
10106.08984	8345.873047	44.224998
10108.69531	8374.139648	44.301998
10111.12598	8402.34082	44.379002
10115.84961	8458.867188	44.532001
10120.19043	8515.391602	44.685001
10122.26074	8543.688477	44.761002
10124.24316	8571.991211	44.838001
10126.1377	8600.298828	44.914001
10129.68945	8656.950195	45.064999
10132.89844	8713.626953	45.216999
10134.39063	8741.986328	45.292
10137.11719	8798.723633	45.443001
10138.37891	8827.123047	45.518002
10139.51465	8855.493164	45.592999
10140.60059	8883.899414	45.667999
10141.58008	8912.291992	45.743
10142.50879	8940.71875	45.818001
10143.35156	8969.149414	45.891998
10144.79102	9026.033203	46.041
10145.375	9054.472656	46.116001
10145.87109	9082.914062	46.189999
10146.30664	9111.380859	46.264
10146.65332	9139.84668	46.338001
10146.93359	9168.333984	46.411999
10147.12305	9196.817383	46.486
10147.23828	9253.785156	46.632999

In [18]:
generate_ballistics_table(375, 0)

x:  0 y: - 0 z:  0
x:  1693.5 y: - -100 z:  4.515999999999843
x:  2395.125 y: - -200 z:  6.387000000000468
x:  2933.25 y: - -300 z:  7.822000000000947
x:  3386.625 y: - -400 z:  9.031000000000434
x:  3786.375 y: - -500 z:  10.096999999999843
x:  4147.875 y: - -600 z:  11.060999999999309
x:  4480.125 y: - -700 z:  11.946999999998818
x:  4789.5 y: - -800 z:  12.77199999999836
x:  5080.125 y: - -900 z:  13.546999999997931
x:  5354.625 y: - -1000 z:  14.278999999997525
x:  5616.0 y: - -1100 z:  14.97599999999714
x:  5865.75 y: - -1200 z:  15.64199999999677
x:  6105.375 y: - -1300 z:  16.280999999996915
x:  6335.625 y: - -1400 z:  16.894999999997665
x:  6558.0 y: - -1500 z:  17.48799999999839
x:  6773.25 y: - -1600 z:  18.06199999999909
x:  6981.75 y: - -1700 z:  18.61799999999977
x:  7184.25 y: - -1800 z:  19.15800000000043
x:  7380.75 y: - -1900 z:  19.68200000000107
x:  7572.75 y: - -2000 z:  20.194000000001697
x:  7759.5 y: - -2100 z:  20.692000000002306
x:  7942.125 y: - -2200 z:  21.1

In [19]:
initial_velocity = 121.3
elevation = 45

print(ballistic_sim(initial_velocity, elevation, 0, .0001))
print(ballistic_sim2(initial_velocity, elevation, 0, .0001))
print(ballistic_sim3(initial_velocity, elevation, 0, .001, False))

(1499.8787286697336, 17.486799999972586)
(969.710060307664, 14.06059999998057)
(np.float64(1499.9816551334366), 17.48799999999839, [array([0, 0]), array([0.08577205, 0.08577205]), array([0.17154411, 0.1715343 ]), array([0.25731616, 0.25728673]), array([0.34308821, 0.34302935]), array([0.42886026, 0.42876216]), array([0.51463232, 0.51448517]), array([0.60040437, 0.60019836]), array([0.68617642, 0.68590174]), array([0.77194847, 0.77159531]), array([0.85772053, 0.85727908]), array([0.94349258, 0.94295303]), array([1.02926463, 1.02861717]), array([1.11503668, 1.1142715 ]), array([1.20080874, 1.19991603]), array([1.28658079, 1.28555074]), array([1.37235284, 1.37117564]), array([1.45812489, 1.45679073]), array([1.54389695, 1.54239602]), array([1.629669  , 1.62799149]), array([1.71544105, 1.71357715]), array([1.8012131, 1.799153 ]), array([1.88698516, 1.88471905]), array([1.97275721, 1.97027528]), array([2.05852926, 2.0558217 ]), array([2.14430131, 2.14135831]), array([2.23007337, 2.22688512]

In [20]:
def deg2EL(degrees):
    return math.radians(degrees) * 1000 - 960

In [21]:
print(calculate_elevation(121.3, 2, 1036, True)['elevation'])
elevation_regression(1036)

68.1


169.83107231155205

In [22]:
rotate_point_clockwise(np.array([-13,-18.4]), 145)

AttributeError: 'numpy.ndarray' object has no attribute 'point'

In [11]:
import matplotlib
import matplotlib.pyplot as plt

for angle in range(10,45):
    plot_points = ballistic_sim3(375, angle, 0, .001)[2]
    x = [point[0] for point in plot_points]
    y = [point[1] for point in plot_points]
    x = x[::10]
    y = y[::10]
    myplot = plt.plot(x, y)
    ax = plt.gca()
    ax.set_xlim([0,8500])
    # test = ax.plot()[0]
    # test.set_data()
    plt.show()

TypeError: ballistic_sim3() missing 1 required positional argument: 'include_drag'

In [27]:
len(x)

4471